# Descarga de imágenes de satélite de Copernicus

### Documentación oficial [aquí](https://documentation.dataspace.copernicus.eu/APIs/OData.html "OData's Documentation") de la API **OData**

https://documentation.dataspace.copernicus.eu/APIs/OData.html

### URL

Para trabajar con la API OData de Copernicus, es importante entender cómo estructurar las consultas y qué opciones están disponibles para filtrar y recuperar datos específicos.

In [1]:
url="""
        https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq 'SENTINEL-2' and 
        ContentDate/Start gt 2024-01-01T00:00:00.000Z and ContentDate/Start lt 2030-12-31T00:10:00.000Z and 
        Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/OData.CSC.DoubleAttribute/Value le 40) and
        Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/OData.CSC.StringAttribute/Value eq 'S2MSI2A') and 
        OData.CSC.Intersects(area=geography'SRID=4326;POLYGON ((-79.6094133704819 22.2004997536145,-79.7167043945783 21.9193972704819,-79.2918319391566 21.8121062463855,-79.1652285307229 22.1468542415663,-79.6094133704819 22.2004997536145))')&$orderby=ContentDate/Start desc"""

**línea 1:** Filtra por nombre == 'SENTINEL-2' --> Las misiones Sentinel-2 se emplean con fines agrometeorológicos, tienen una frecuencia de revisita de 10 días, pero como son 2 satélites gemelos se reduce el sensoramiento del mismo sitio a 5 días

**línea 2:** Filtra las imágenes entre el 01/01/2024 y el 31/12/2024 (aquí se usa formato americano YY-mm-dd) --> luego más adelante se organizaran las imágenes en orden descendente por lo que las más recientes quedarán en el top

**línea 3:** Filtra por nubosidad <= 40 %

**línea 4:** Filtra todos los productos que coincidan con 'S2MSI2A' --> S2MSI2A sensor de imagen multiespectral (MSI), nivel 2A se encuentran ortorectificadas con niveles de reflectancia por debajo de la atmósfera ([BOA](https://www.gisandbeers.com/lo-deberias-saber-imagenes-sentinel-2/))

**línea 5:** Filtra aquellas imágenes que coincida con un área de interés, usa formato well-know (el ejemplo de la url: abarca un pedazo de la provincia de Sancti Spíritus) y termina la línea ordenando los resultados del más reciente al más antiguo (orden descendente)

### Request, petición 1

In [2]:
import requests
import pandas as pd

df = pd.DataFrame()

#proxies = {'http':'http://[user]:[password]@[proxy]:[port]'}  # usar si se tiene proxy
#req = requests.get(proxies=proxies, url=url)                  # usar si se tiene proxy
req = requests.get(url=url)                                    # usar si no se tiene proxy
url_current = req.url
json = req.json()

# Print only specific columns
if len(json) != 1:
    df = pd.DataFrame(json['value'])
if len(df) != 0:
    if '@odata.nextLink' in json.keys():
        url_next = json['@odata.nextLink']
    columns_to_print = ['Id', 'Name','S3Path','GeoFootprint'] 
df[columns_to_print].head(3) # ojo aquí en la visualización te estoy sesgando la metadata a solo 4 campos que son importantes para que te ubiques)

,Id,Name,S3Path,GeoFootprint
0,9af6dbb1-56de-43b0-b890-0e445d79bc62,S2B_MSIL2A_20240911T160509_N0511_R054_T17QPE_2...,/eodata/Sentinel-2/MSI/L2A/2024/09/11/S2B_MSIL...,"{'type': 'Polygon', 'coordinates': [[[-79.2179..."
1,4caf5409-b9c1-4d51-9363-d0219d488254,S2A_MSIL2A_20240906T160511_N0511_R054_T17QPE_2...,/eodata/Sentinel-2/MSI/L2A/2024/09/06/S2A_MSIL...,"{'type': 'Polygon', 'coordinates': [[[-79.2105..."
2,77c25c2e-ab21-45fe-a009-05a01cb9cf24,S2A_MSIL2A_20240903T155521_N0511_R011_T17QPE_2...,/eodata/Sentinel-2/MSI/L2A/2024/09/03/S2A_MSIL...,"{'type': 'Polygon', 'coordinates': [[[-79.8503..."


**['Id', 'Name','S3Path','GeoFootprint']**, pero te muestro todos las disponibles

In [3]:
[ii for ii in df.columns]

['@odata.mediaContentType',
 'Id',
 'Name',
 'ContentType',
 'ContentLength',
 'OriginDate',
 'PublicationDate',
 'ModificationDate',
 'Online',
 'EvictionDate',
 'S3Path',
 'Checksum',
 'ContentDate',
 'Footprint',
 'GeoFootprint']

In [4]:
df.head(3)

,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,9af6dbb1-56de-43b0-b890-0e445d79bc62,S2B_MSIL2A_20240911T160509_N0511_R054_T17QPE_2...,application/octet-stream,807690091,2024-09-11T22:12:16.000000Z,2024-09-11T22:24:36.278143Z,2024-09-11T22:25:32.807307Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2024/09/11/S2B_MSIL...,"[{'Value': '560e21b10b60ca8f28d52d6634060005',...","{'Start': '2024-09-11T16:05:09.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-79.217945379085...,"{'type': 'Polygon', 'coordinates': [[[-79.2179..."
1,application/octet-stream,4caf5409-b9c1-4d51-9363-d0219d488254,S2A_MSIL2A_20240906T160511_N0511_R054_T17QPE_2...,application/octet-stream,792977900,2024-09-06T23:28:59.000000Z,2024-09-06T23:40:11.021501Z,2024-09-06T23:41:07.757129Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2024/09/06/S2A_MSIL...,"[{'Value': 'e50b860c81a120b072321778142042b7',...","{'Start': '2024-09-06T16:05:11.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-79.210508664189...,"{'type': 'Polygon', 'coordinates': [[[-79.2105..."
2,application/octet-stream,77c25c2e-ab21-45fe-a009-05a01cb9cf24,S2A_MSIL2A_20240903T155521_N0511_R011_T17QPE_2...,application/octet-stream,873226397,2024-09-03T23:13:27.000000Z,2024-09-03T23:22:25.320100Z,2024-09-03T23:23:24.037963Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2024/09/03/S2A_MSIL...,"[{'Value': 'ace3a6edd52acb0596676d33d9ed5005',...","{'Start': '2024-09-03T15:55:21.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-79.850338734664...,"{'type': 'Polygon', 'coordinates': [[[-79.8503..."


### Identificador de la imagen más reciente

In [5]:
descarga = df.iloc[0,:]
descarga['Id']

'9af6dbb1-56de-43b0-b890-0e445d79bc62'

### Crearse cuenta en Copernicus

Luego de creada la cuenta puede seguir el siguiente tutorial para usar los **OAuth client** 

[tutorial OAuth client](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html) --> (https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html)

### Petición request 2

In [6]:
import requests

# Detalles de la solicitud
url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
data = {
    'client_id': 'cdse-public',
    'username': 'odelgado@utem.cl',  # Reemplaza con tu username
    'password': 'fN6"8N7983EBUg*',  # Reemplaza con tu password
    'grant_type': 'password'
}

# Realiza la solicitud POST
response = requests.post(url, data=data)

# Si la respuesta es exitosa, extrae el token
if response.status_code == 200:
    response_json = response.json()
    access_token = response_json.get('access_token')
    if access_token:
        print(f"ACCESS_TOKEN={access_token}")
    else:
        print("No se encontró el token de acceso en la respuesta.")
else:
    print(f"Error en la solicitud: {response.status_code}")


ACCESS_TOKEN=eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJYVUh3VWZKaHVDVWo0X3k4ZF8xM0hxWXBYMFdwdDd2anhob2FPLUxzREZFIn0.eyJleHAiOjE3MjYyNjU5MjUsImlhdCI6MTcyNjI2NTMyNSwianRpIjoiODRkNDkzZDAtNjcwZi00OWExLTk0YTUtYjY4ZjY2MTg1MjQ0IiwiaXNzIjoiaHR0cHM6Ly9pZGVudGl0eS5kYXRhc3BhY2UuY29wZXJuaWN1cy5ldS9hdXRoL3JlYWxtcy9DRFNFIiwiYXVkIjpbIkNMT1VERkVSUk9fUFVCTElDIiwiYWNjb3VudCJdLCJzdWIiOiI2Y2JjMjdlOC00Y2Y0LTRlZTQtYjgwMi1jZTQ3YjY4NDM2MmIiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJjZHNlLXB1YmxpYyIsInNlc3Npb25fc3RhdGUiOiI3Njc3NzZiNy1jMTkxLTRlMWYtYmU1Mi01OTNiOGI0M2IxNzgiLCJhbGxvd2VkLW9yaWdpbnMiOlsiaHR0cHM6Ly9sb2NhbGhvc3Q6NDIwMCIsIioiLCJodHRwczovL3dvcmtzcGFjZS5zdGFnaW5nLWNkc2UtZGF0YS1leHBsb3Jlci5hcHBzLnN0YWdpbmcuaW50cmEuY2xvdWRmZXJyby5jb20iXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLWNkYXMiLCJjb3Blcm5pY3VzLWdlbmVyYWwiXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJzY29

### Product Download

Seguir los pasos del [tutorial](https://documentation.dataspace.copernicus.eu/APIs/OData.html "OData's Documentation") en la sección **Product Download**.

In [7]:
descarga['Name'] # nombre del ráster que se descargará

'S2B_MSIL2A_20240911T160509_N0511_R054_T17QPE_20240911T211003.SAFE'

**URL** para realizar la descarga, se le pasa el Identificador (**descarga['Id']**)

In [8]:
url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({descarga['Id']})/$value"
url

'https://zipper.dataspace.copernicus.eu/odata/v1/Products(9af6dbb1-56de-43b0-b890-0e445d79bc62)/$value'

### Request, petición 3

Para la petición 1 no necesitamos usar los credenciales y pudimos listar las últimas imágenes disponibles.

Para la petición 2 usamos los credenciales para obtener el token de acceso

La petición 3 la usaremos para descargar la imagen del satélite

In [9]:
import requests

headers = {"Authorization": f"Bearer {access_token}"}                       # este paso es crítico, si no se usa el token de acceso, el sistema te deniega
#proxies = {'http':'http://[user]:[password]@[proxy]:[port]'}               # usar si se tiene proxy
session = requests.Session()
session.headers.update(headers)
#response = session.get(url, headers=headers, stream=True, proxies=proxies) # usar si se tiene proxy
response = session.get(url, headers=headers, stream=True)                   # usar si no se tiene proxy
with open(f"{descarga['Name']}.zip", "wb") as file:                                     # ponerle un nombre al comprimido, pudiera ser el alguna combinación con descarga['Name'] o alguno de interés propio
    for chunk in response.iter_content(chunk_size=8192):
        if chunk:
            file.write(chunk)
print(url)
print(response)

https://zipper.dataspace.copernicus.eu/odata/v1/Products(9af6dbb1-56de-43b0-b890-0e445d79bc62)/$value
<Response [200]>
